In [1]:
1+1

2

In [2]:
%pip install arxiv

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_community.tools import WikipediaQueryRun


In [4]:
from langchain_community.utilities import WikipediaAPIWrapper

In [49]:
from langchain_core.tools import Tool
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun

wiki_wrapper = WikipediaAPIWrapper(
    top_k_results=1,
    doc_content_chars_max=200
)

wiki_runner = WikipediaQueryRun(api_wrapper=wiki_wrapper)

def wikipedia_search(query: str) -> str:
    return wiki_runner.run(query)

safe_wikipedia_tool = Tool(
    name="wikipedia_search",   # Gemini-safe
    description="Search Wikipedia for factual information",
    func=wikipedia_search,
)


In [6]:
%pip install wikipedia

Note: you may need to restart the kernel to use updated packages.


In [7]:
# https://docs.langchain.com/oss/python/langchain/overview

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter


loader=WebBaseLoader("https://docs.langchain.com/oss/python/langchain/overview")
docs=loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(docs)

vectordb = FAISS.from_documents(documents, HuggingFaceEmbeddings())

retriever=vectordb.as_retriever()



d:\GenAI\LangChain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
USER_AGENT environment variable not set, consider setting it to identify your requests.
C:\Users\DELL\AppData\Local\Temp\ipykernel_3824\1205357829.py:14: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  vectordb = FAISS.from_documents(documents, HuggingFaceEmbeddings())
C:\Users\DELL\AppData\Local\Temp\ipykernel_3824\1205357829.py:14: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in Lang

In [8]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000016C63FF81D0>, search_kwargs={})

In [48]:
# from langchain_core.tools import create_retriever_tool
# retriever_tool = create_retriever_tool(
#     retriever=retriever,
#     name="LangChain Documentation Retriever",
#     description="Useful for when you need to answer questions about LangChain's Python documentation. "
#     "Input should be a fully formed question."
# )

from langchain_core.tools import Tool

def retrieve_docs(query: str) -> str:
    docs = retriever.get_relevant_documents(query)
    return "\n".join(d.page_content for d in docs)

safe_retriever_tool = Tool(
    name="document_retrieval",
    description="Retrieve relevant documents from the knowledge base",
    func=retrieve_docs,
)



In [43]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv_wrapper)

from langchain_core.tools import Tool

def arxiv_search(query: str) -> str:
    return arxiv_tool.run(query)

safe_arxiv_tool = Tool(
    name="arxiv_search",  # Gemini-safe name
    description="Search arXiv for relevant research papers and return summaries.",
    func=arxiv_search,
)



In [50]:
tools = [safe_wikipedia_tool, safe_retriever_tool, safe_arxiv_tool]

In [12]:
from dotenv import load_dotenv
load_dotenv()
import os
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")


In [16]:
# prompt
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use tools when necessary."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}")
])



In [34]:

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import Tool
from langchain_classic.agents import AgentExecutor, create_tool_calling_agent


In [67]:
agent=create_tool_calling_agent(
    llm=ChatGoogleGenerativeAI(model="models/gemini-2.5-flash"),
    tools=tools,
    prompt=prompt,
)

In [68]:
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=tools,
    verbose=True,
)

In [69]:
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['input'], optional_variables=['agent_scratchpad'], input_types={'agent_scratchpad': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')

In [78]:
agent_executor.invoke({"input": "Summarise langchain and its concepts."})



> Entering new AgentExecutor chain...

Invoking: `wikipedia_search` with `{'langchain': 'langchain'}`


Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain[{'type': 'text', 'text': 'LangChain is a software framework designed to facilitate the integration of large language models (LLMs) into various applications. It serves as a language model integration framework.', 'extras': {'signature': 'CiQBcsjafJ9KZEwQKn8IrNTJ9HjO07HDnt8892px1PjrWpMhqjYKiQEBcsjafEDHCAzYByTqOeXZrKcc3cEjtxh2Pzc+Bk+pruUdA+lGMcSvh3kP2/KvFELXQeSPonBN5l91rZ18iS+0QkL778Fi1cXHqaqMbItXu6kWORGAY/wKEBzFWf04OHDuT6DZtm3jb9XKVikLqtrd6vFalzX0Tq0npqc2SYGwEx7wCYvcggDhtgrvAQFyyNp8Nedqh8pdyQufsRvlYyo4LQG630MXGvKz7jt+B+1iBI2Qa89Q8xVapOlxLUOfqJsJ2MbFX7uUBew3pWjW55peBjVgnocDuKg+Hhlf5KFWJ5ly9eVTwGYQnky/gv35lL4R7+qyxXlmZk9ZFrxHA0SS+S7luRx6kJc0UZeESG0Qan3HFGVjhsp8M6O6ZsyeVkY6MGPwpwUYpD0Xgzp

{'input': 'Summarise langchain and its concepts.',
 'output': [{'type': 'text',
   'text': 'LangChain is a software framework designed to facilitate the integration of large language models (LLMs) into various applications. It serves as a language model integration framework.',
   'extras': {'signature': 'CiQBcsjafJ9KZEwQKn8IrNTJ9HjO07HDnt8892px1PjrWpMhqjYKiQEBcsjafEDHCAzYByTqOeXZrKcc3cEjtxh2Pzc+Bk+pruUdA+lGMcSvh3kP2/KvFELXQeSPonBN5l91rZ18iS+0QkL778Fi1cXHqaqMbItXu6kWORGAY/wKEBzFWf04OHDuT6DZtm3jb9XKVikLqtrd6vFalzX0Tq0npqc2SYGwEx7wCYvcggDhtgrvAQFyyNp8Nedqh8pdyQufsRvlYyo4LQG630MXGvKz7jt+B+1iBI2Qa89Q8xVapOlxLUOfqJsJ2MbFX7uUBew3pWjW55peBjVgnocDuKg+Hhlf5KFWJ5ly9eVTwGYQnky/gv35lL4R7+qyxXlmZk9ZFrxHA0SS+S7luRx6kJc0UZeESG0Qan3HFGVjhsp8M6O6ZsyeVkY6MGPwpwUYpD0XgzpLuS7kr6zhg6+Jp56kKtkBN88VbildwdRTBV0DfuB2X3Lh1dbr0MrkItPzzvmV3xIJj0CYmumblWktQFMi1/TWkseHQOlOla8nZ/kjPBNR'},
   'index': 0}]}